预处理第二步

In [1]:
from __future__ import annotations

import io
import shutil
import math
from pathlib import Path

import requests
# import tensorflow as tf
import pandas as pd
import numpy as np

from datetime import datetime, date
from rich.progress import track
from PIL import Image
from IPython.display import display


In [5]:
data = pd.read_csv("dataset.csv", low_memory=False)
data.drop(labels=["Unnamed: 0"], axis=1, inplace=True)
data.head()

,rest name,address,price,star,reviews count,class,author,from,elite,author friends,...,photo_1 link,photo_2 link,photo_3 link,photo_4 link,photo_5 link,photo_6 link,photo_7 link,photo_8 link,photo_9 link,photo_10 link
0,101 Beer Kitchen,"7509 Sawmill Rd\nDublin, OH 43016\n",$$,4.0,746,American (New)-Gastropubs-Breakfast & Brunch,Keith B.,"Powell, OH",2022.0,407,...,https://s3-media0.fl.yelpcdn.com/bphoto/dNiwDl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101 Beer Kitchen,"7509 Sawmill Rd\nDublin, OH 43016\n",$$,4.0,746,American (New)-Gastropubs-Breakfast & Brunch,Norma O.,"Shawnee Hills, OH",NaN,14,...,https://s3-media0.fl.yelpcdn.com/bphoto/jz66xj...,https://s3-media0.fl.yelpcdn.com/bphoto/L-saam...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101 Beer Kitchen,"7509 Sawmill Rd\nDublin, OH 43016\n",$$,4.0,746,American (New)-Gastropubs-Breakfast & Brunch,Carrie A.,"Fairborn, OH",2022.0,69,...,https://s3-media0.fl.yelpcdn.com/bphoto/oV-qJw...,https://s3-media0.fl.yelpcdn.com/bphoto/fSuMPd...,https://s3-media0.fl.yelpcdn.com/bphoto/05mCVH...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101 Beer Kitchen,"7509 Sawmill Rd\nDublin, OH 43016\n",$$,4.0,746,American (New)-Gastropubs-Breakfast & Brunch,Jeppy L.,"Columbus, OH",2022.0,96,...,https://s3-media0.fl.yelpcdn.com/bphoto/eqKst8...,https://s3-media0.fl.yelpcdn.com/bphoto/HM9YAh...,https://s3-media0.fl.yelpcdn.com/bphoto/UguJBW...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101 Beer Kitchen,"7509 Sawmill Rd\nDublin, OH 43016\n",$$,4.0,746,American (New)-Gastropubs-Breakfast & Brunch,Jennifer C.,"Columbus, OH",2022.0,60,...,https://s3-media0.fl.yelpcdn.com/bphoto/Ii_y29...,https://s3-media0.fl.yelpcdn.com/bphoto/_XthnY...,https://s3-media0.fl.yelpcdn.com/bphoto/eRmrs9...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 图片有用性猜测
# RAW_IMAGE_DIR = Path("../input/openlabyelp-preprocess-1/raw_images/images")  # 整理前图片存放目录

# index_max = data[data["useful"] == data["useful"].max()].index.tolist()[0]
# sample = data.sample(1)
# index_sample = sample.index.tolist()[0]
# print(sample["useful"])

# for filename in (RAW_IMAGE_DIR / str(index_max)).iterdir():
#     image = Image.open(filename)
#     display(image)

# print("=" * 80)

# for filename in (RAW_IMAGE_DIR / str(index_sample)).iterdir():
#     image = Image.open(filename)
#     display(image)

In [2]:
def to_int(x, ignore=True):
    try:
        return int(x)
    except ValueError:
        if ignore:
            return 0
        else:
            raise

In [7]:
data["author friends"] = data["author friends"].apply(to_int)
data["author reviews"] = data["author reviews"].apply(to_int)
print(len(data))

63384


In [8]:
error_rows = data[data["author friends"].apply(lambda x: len(str(x)) > 5)]
error_indices = error_rows.index.tolist()
# data.drop(index=error_indices, inplace=True)
print(error_indices)
error_rows
# 这是异常值

error_rows2 = data[data["photo_1 link"].apply(lambda x: x is np.nan)]
error_indices2 = error_rows2.index.tolist()
# data.drop(index=error_indices, inplace=True)
print(error_indices2)
error_rows2

error_rows3 = data[data["price"].apply(lambda x: x != len(x) * x[0])]
error_indices3 = error_rows3.index.tolist()
# data.drop(index=error_indices, inplace=True)
print(error_indices3)
error_rows3

[]
[]
[7521, 13663, 48553, 59259, 59514]


,rest name,address,price,star,reviews count,class,author,from,elite,author friends,...,photo_1 link,photo_2 link,photo_3 link,photo_4 link,photo_5 link,photo_6 link,photo_7 link,photo_8 link,photo_9 link,photo_10 link
7521,Burger Baron,"3909 37 Avenue\nWhitecourt, AB T7S 0C3\nCanada\n",Burgers,4.5,2,NaN,Lexter C.,"Calgary, Canada",NaN,5,...,https://s3-media0.fl.yelpcdn.com/bphoto/ARFe5t...,https://s3-media0.fl.yelpcdn.com/bphoto/CiltIj...,https://s3-media0.fl.yelpcdn.com/bphoto/PMayk-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13663,Edo Japan,"102 - 3815 Caxton Street\nWhitecourt, AB T7S 0...",Japanese,4.5,4,NaN,Nina W.,"Calgary, Canada",NaN,57,...,https://s3-media0.fl.yelpcdn.com/bphoto/t_FiPx...,https://s3-media0.fl.yelpcdn.com/bphoto/0gnuKJ...,https://s3-media0.fl.yelpcdn.com/bphoto/16MRGR...,https://s3-media0.fl.yelpcdn.com/bphoto/EPzLIu...,https://s3-media0.fl.yelpcdn.com/bphoto/3CepSS...,https://s3-media0.fl.yelpcdn.com/bphoto/vJI47P...,https://s3-media0.fl.yelpcdn.com/bphoto/N0-5cD...,NaN,NaN,NaN
48553,Spring Sun Restaurant,"4816 53 Avenue\nBarrhead, AB T7N 1H2\nCanada\n",Chinese,3.5,2,NaN,Vad Z.,"Edmonton, Canada",NaN,0,...,https://s3-media0.fl.yelpcdn.com/bphoto/LoEp1l...,https://s3-media0.fl.yelpcdn.com/bphoto/XYzZ-d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59259,Violet Mae's Bakery,"5107 50th Avenue\nWhitecourt, AB T7S 1S8\nCana...",Bakeries,4.0,1,NaN,Glen B.,"Edmonton, Canada",2022.0,25,...,https://s3-media0.fl.yelpcdn.com/bphoto/g4b2fE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59514,West of the 5th Distillery,"5110 51 St\nBarrhead, AB T7N 1A1\nCanada\n",Burgers,4.0,1,NaN,John A.,"Spruce Grove, Canada",2022.0,2,...,https://s3-media0.fl.yelpcdn.com/bphoto/i3AOE2...,https://s3-media0.fl.yelpcdn.com/bphoto/4-e5tE...,https://s3-media0.fl.yelpcdn.com/bphoto/GrrJvN...,https://s3-media0.fl.yelpcdn.com/bphoto/kaY0nv...,https://s3-media0.fl.yelpcdn.com/bphoto/KBVllt...,NaN,NaN,NaN,NaN,NaN


In [9]:
# 去除 useful < 1 的行
zero_useful_indices = data[(data["useful"] < 1)].index.tolist()
len(zero_useful_indices), len(data)

(0, 63384)

In [10]:
xxx=data["elite"].apply(lambda x: 1 if x>0 else 0)
xxx[8:20]

8     0
9     1
10    1
11    1
12    0
13    1
14    0
15    0
16    0
17    0
18    1
19    0
Name: elite, dtype: int64

重新整理图片（去除的图片类别）

In [ ]:
# Linux only
!rm -rf images

RAW_IMAGE_DIR = Path("../input/raw-image/raw_images/images")  # 整理前图片存放目录
IMAGE_DIR = Path("images")  # 整理后图片存放目录
ERROR_INDICES = set(error_indices + error_indices2 + error_indices3 + zero_useful_indices)

# 图片整理
IMAGE_DIR.mkdir(exist_ok=True, parents=True)
total = len(list(RAW_IMAGE_DIR.iterdir()))  # data 长度已经改变，不能用 len(data)
from_dirs = [RAW_IMAGE_DIR / str(name) for name in range(total)]  # 按数字排序的图片目录
from_to = {}
to_dir = 0
for from_dir in track(from_dirs, description="图片整理", refresh_per_second=1):
    if int(from_dir.name) in ERROR_INDICES:  # 跳过错误的数据
        continue

    if not from_dir.exists():  # 目录不存在
        continue

    if not list(from_dir.iterdir()):  # 空文件夹
        continue

    shutil.copytree(from_dir, IMAGE_DIR / str(to_dir))
    from_to[int(from_dir.name)] = to_dir  # 保存映射关系

    to_dir += 1

with open("from_to.txt", "w", encoding="utf8") as f:
    f.writelines(f"{k} -> {v}\n" for k, v in from_to.items())

In [ ]:
# 调整图片大小
from_dirs = list(IMAGE_DIR.iterdir())
for from_dir in track(from_dirs, description="大小调整", refresh_per_second=1):
    for filename in from_dir.iterdir():
        image = Image.open(filename)
        image.load()
        image = image.resize((224, 224))
        image.save(filename)

In [12]:
print(f"去除无图片数据前有 {len(data)} 条数据")

# keys = {i for i in from_to.keys()}
# droped = set(data.index) - keys
# print(f"去除 {len(droped)} 条数据")

# data = data.iloc[list(keys)]
print(f"去除无图片数据后剩余 {len(data)} 条数据")

data.reset_index(inplace=True, drop=True)

去除无图片数据前有 63384 条数据
去除无图片数据后剩余 63384 条数据


In [ ]:
# 检查是否正确
#sample = data.sample(n=1)["photo_1 link"]
#url = sample.item()
#index = sample.index.item()
index = len(data) - 1
sample = data.iloc[index]
url = sample["photo_1 link"]

resp = requests.get(url)
image1 = Image.open(io.BytesIO(resp.content)).resize((512, 512))
display(image1)

image_dir = IMAGE_DIR / str(index)
for file in image_dir.iterdir():
    display(Image.open(file))

数据处理

In [13]:
def to_date(date_string, format, error=False):
    if error:
        return datetime.strptime(date_string, format)

    try:
        return datetime.strptime(date_string, format)
    except (ValueError, TypeError):
        pass

def fillna_with_mean(column, dtype):
    column.fillna(column.mean(), inplace=True)
    column = column.astype(dtype)
    return column

def fillna_with_zero(column):
    column.fillna(0, inplace=True)
    column = column.astype("int")
    return column

In [14]:
DATA_DATE = date(2022, 10, 10)  # 数据获取时间

new_data = pd.DataFrame()

#data.drop(index=101957, inplace=True)
data.to_csv("dataset_clean.csv")

# star 餐厅星级
new_data["star"] = fillna_with_mean(data["star"], "float32")

# price 餐厅平均消费
new_data["price"] = data["price"].apply(lambda x: len(x))

# reviews count 餐厅评论数量
new_data["reviews count"] = fillna_with_mean(data["reviews count"], "int")

# elite 用户是否精英
new_data["elite"] = data["elite"].apply(lambda x: 1 if x > 0 else 0)

# author friends  # 用户朋友数量
new_data["author friends"] = fillna_with_mean(data["author friends"], "int")

# author reviews  # 用户评论总数
new_data["author reviews"] = fillna_with_mean(data["author reviews"], "int")

# author photos  # 用户上传过的照片总数
new_data["author photos"] = fillna_with_mean(data["author photos"], "int")

# join date(day delta)  # 用户加入 Yelp 距离 DATA_DATE 的日期（以天计）
# 原始数据为 (年, 月)，这里以每个月 1 号补足，即 (年, 月, 1)
join_date = data["join date"].apply(to_date, format="%B %Y", error=False)
join_date.fillna(join_date.mean(), inplace=True)  # 用日期均值填充空白值
new_data["author join date(day delta)"] = join_date.apply(lambda x: (DATA_DATE - x.date()).days)
new_data["author join date(day delta)"] = new_data["author join date(day delta)"].apply(lambda x: 0 if x < 0 else x)


# author useful votes  # 用户所有的 useful 投票，抓取失败，全是0
#new_data["author useful votes"] = fillna_with_mean(data["author useful votes"], "int")

# author funny votes  # 用户所有的 funny 投票
new_data["author funny votes"] = fillna_with_mean(data["author funny votes"], "int")

# author cool votes  # 用户所有的 cool 投票
new_data["author cool votes"] = fillna_with_mean(data["author cool votes"], "int")

# author rating distribution  # 用户评分分布
new_data["author 5 rating distribution"] = fillna_with_mean(data["rating_distribution_5"], "int")
new_data["author 4 rating distribution"] = fillna_with_mean(data["rating_distribution_4"], "int")
new_data["author 3 rating distribution"] = fillna_with_mean(data["rating_distribution_3"], "int")
new_data["author 2 rating distribution"] = fillna_with_mean(data["rating_distribution_2"], "int")
new_data["author 1 rating distribution"] = fillna_with_mean(data["rating_distribution_1"], "int")

# review date(day delta)  # 评论距离抓取时间（以日为单位）
review_date = data["review date"].apply(to_date, format="%m/%d/%Y", error=False)
review_date.fillna(review_date.mean(), inplace=True)  # 用日期均值填充空白值
new_data["review date(day delta)"] = review_date.apply(lambda x: (DATA_DATE - x.date()).days)

# review rating  # 评论评分
new_data["review rating"] = fillna_with_mean(data["review rating"], "int")

# review photos  # 评论的照片数量
new_data["review photos"] = fillna_with_mean(data["totalPhotos"], "int")

# review useful  # 评论 useful
new_data["review useful original"] = fillna_with_mean(data["useful"], "int")

# review funny
#new_data["review funny"] = fillna_with_mean(data["funny"], "int")

# review cool
#new_data["review cool"] = fillna_with_mean(data["cool"], "int")

# review text
new_data["review text"] = data["review text"]

# review reply date(day delta)
reply_date = data["businessOwnerRepliesdate"].apply(to_date, format="%m/%d/%Y", error=False)
data_datetime = datetime(DATA_DATE.year, DATA_DATE.month, DATA_DATE.day)
reply_date.fillna(data_datetime, inplace=True)  # 用 0 表示无回复
new_data["review reply date(day delta)"] = reply_date.apply(lambda x: (DATA_DATE - x.date()).days)
new_data["review reply date(day delta)"] = new_data["review reply date(day delta)"].apply(lambda x: 0 if x < 0 else x)
# review is updated
#new_data["from"] = data["isUpdated"].apply(bool)

In [ ]:
#new_data.iloc[101957]

In [15]:
new_data["review useful"] = new_data["review useful original"].apply(lambda x: math.log(x+1))

In [16]:
new_data["review useful"].describe()
new_data["review useful original"].describe()

count    63384.000000
mean         3.427300
std          7.137581
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max        327.000000
Name: review useful original, dtype: float64

In [17]:
data["useful"].describe()

count    63384.000000
mean         3.427300
std          7.137581
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max        327.000000
Name: useful, dtype: float64

In [ ]:
# (new_data["review date(day delta)"]).apply(lambda x: x / 365 if x > 365 else 1).describe()

In [ ]:
# new_data["review useful"] = new_data["review useful"].apply(lambda x: x if x < 100 else 100)
# new_data["review useful"] = (new_data["review useful"] / (new_data["review date(day delta)"]).apply(lambda x: x / 365 if x > 365 else 1))

In [18]:
new_data["review useful"]

0        0.693147
1        0.693147
2        1.098612
3        0.693147
4        0.693147
           ...   
63379    1.609438
63380    0.693147
63381    0.693147
63382    0.693147
63383    0.693147
Name: review useful, Length: 63384, dtype: float64

保存数据

In [19]:
new_data.to_csv("new_dataset.csv")
!tar -czf images.tar.gz images/ --remove-files

tar: : Couldn't visit directory: No such file or directory
tar: : Couldn't visit directory: No such file or directory
tar: Error exit delayed from previous errors.


In [20]:
test = pd.read_csv("new_dataset.csv", low_memory=False)
test.dtypes

Unnamed: 0                        int64
star                            float64
price                             int64
reviews count                     int64
elite                             int64
author friends                    int64
author reviews                    int64
author photos                     int64
author join date(day delta)       int64
author funny votes                int64
author cool votes                 int64
author 5 rating distribution      int64
author 4 rating distribution      int64
author 3 rating distribution      int64
author 2 rating distribution      int64
author 1 rating distribution      int64
review date(day delta)            int64
review rating                     int64
review photos                     int64
review useful original            int64
review text                      object
review reply date(day delta)      int64
review useful                   float64
dtype: object